In [1]:
import pandas as pd
import mysql.connector
import gradio as gr

c:\Users\tyler\miniconda3\envs\general\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# TODO: Change these according to your setup
host = "localhost"
user = "root"
password = "YOUR PASSWORD"
database = "University"

In [3]:
def connect_to_database():
    return mysql.connector.connect(
        host = host,
        user = user,
        password = password,
        database = database
    )

def query(sql,params=None):
    db = None
    cursor = None
    try:
        db = connect_to_database()
        cursor = db.cursor()
        cursor.execute(sql,params or ())
        result = cursor.fetchall()
        column_names = [column[0] for column in cursor.description]
        return pd.DataFrame(result, columns=column_names)
    except Exception as e:
        return pd.DataFrame(data={"Error Message":[e]})
    finally:
        # close resources if they were opened
        if cursor:
            cursor.close()
        if db and db.is_connected():
            db.close()

def procedure(proc,params=None):
    db = None
    cursor = None
    try:
        db = connect_to_database()
        cursor = db.cursor()
        cursor.callproc(proc,params or ())
        result_cursor = list(cursor.stored_results())[0]  # just grab the cursor of the first result set
        result = result_cursor.fetchall()
        column_names = [column[0] for column in result_cursor.description]
        return pd.DataFrame(result, columns=column_names)
    except Exception as e:
        return pd.DataFrame(data={"Error Message":[e]})
    finally:
        if cursor:
            cursor.close()
        if db and db.is_connected():
            db.close()

In [4]:
def df_to_html(df):
    return df.to_html(classes="table table-striped", index=False)

In [5]:
# sql queries
avg_crs_depth_prereq = "WITH course_depth_and_prereqs AS ( WITH RECURSIVE prereq_path_length AS ( SELECT P.Crs_id AS start_crs_id, P.Prereq_crs_id AS end_crs_id, 1 AS path_length FROM Prerequisite AS P UNION ALL SELECT PPL.start_crs_id AS start_crs_id, P.Prereq_crs_id AS end_crs_id, (PPL.path_length + 1) AS path_length FROM prereq_path_length AS PPL, Prerequisite AS P WHERE PPL.end_crs_id = P.Crs_id ) SELECT D.Dept_code, C.Crs_num, C.Crs_name, (CASE WHEN MAX(PPL.path_length) IS NULL THEN 0 ELSE MAX(PPL.path_length) END) AS depth, COUNT(DISTINCT PPL.end_crs_id) AS prerequisites FROM Course AS C JOIN Department D ON C.Dept_id = D.Dept_id LEFT JOIN prereq_path_length AS PPL ON C.Crs_id = PPL.start_crs_id GROUP BY C.Crs_id ) SELECT Dept_code, AVG(depth), AVG(prerequisites) FROM course_depth_and_prereqs GROUP BY Dept_code;"
five_lowest_passing_rate = "SELECT D.Dept_code department, C.Crs_num course_num, C.Crs_name course_name, COUNT(DISTINCT FG.Sect_id) num_sections, COUNT(FG.Stud_id) num_students, ROUND(SUM(CASE WHEN FG.grade_point >= 1.0 THEN 1 ELSE 0 END)/COUNT(*),2) passing_rate FROM COURSE C, Section S, Department D, Final_Grade FG WHERE C.Crs_id = S.Crs_id AND C.Dept_id = D.Dept_id AND S.Sect_id = FG.Sect_id GROUP BY C.Crs_id HAVING COUNT(FG.Stud_id) >= 20 ORDER BY passing_rate ASC LIMIT 5;"
five_hardest_prof = "SELECT P.Prof_id id, P.Prof_fname first_name, P.Prof_lname last_name, COUNT(DISTINCT FG.Sect_id) num_sections, COUNT(FG.Stud_id) num_students, ROUND(AVG(FG.overall_grade),3) average_grade, ROUND(AVG(FG.grade_point),2) average_grade_point FROM Professor P, Section S, Final_Grade FG WHERE P.Prof_id = S.Prof_id AND S.Sect_id = FG.Sect_id GROUP BY P.Prof_id HAVING COUNT(FG.Stud_id) >= 20 ORDER BY average_grade ASC LIMIT 5;"
rel_grade_TA = "WITH TA_counts AS ( SELECT Stud_id, COUNT(DISTINCT Sect_id) as times_TAd FROM TA GROUP BY Stud_id ) SELECT CASE WHEN TC.times_TAd > 5 THEN 'Frequent TA (>5 sections)' WHEN tc.times_TAd IS NOT NULL THEN 'Occasional TA (1-5 sections)' ELSE 'Never TA' END TA_frequency, COUNT(DISTINCT s.Stud_id) student_count, ROUND(AVG(S.Stud_GPA),2) average_GPA, ROUND(MIN(s.Stud_GPA),2) minimum_GPA, ROUND(MAX(s.Stud_GPA),2) maximum_GPA FROM Student S LEFT JOIN TA_counts TC ON S.Stud_id = TC.Stud_id WHERE S.Stud_GPA IS NOT NULL GROUP BY TA_frequency ORDER BY average_GPA DESC;"
avg_gpa_by_major = "SELECT D.Dept_name AS department, AVG(S.Stud_GPA) AS average_gpa FROM Major_Minor AS MM, Department AS D, Student AS S WHERE MM.Stud_id = S.Stud_id AND MM.Dept_id = D.Dept_id AND MM.type = 'Major' GROUP BY D.Dept_name ORDER BY average_gpa DESC;"
stud_perf_by_quarter = "SELECT Sect_quarter, ROUND(AVG(fg.grade_point), 3) as avg_grade_point, ROUND(AVG(fg.overall_grade * 100), 2) as avg_percentage FROM Section S, Final_Grade FG WHERE S.Sect_id = FG.Sect_id GROUP BY Sect_quarter ORDER BY Sect_quarter;"

# option => function
opt_fnc = {
    "Custom query (be nice!)": lambda **kwargs: df_to_html(query(kwargs["sql"])),
    "Get student transcript": lambda **kwargs: df_to_html(procedure("GetTranscript",(kwargs["stud_id"],))),
    "Get professor schedule": lambda **kwargs: df_to_html(procedure("GetProfessorSchedule",(kwargs["prof_id"],kwargs["year"],kwargs["quarter"]))),
    "Get course prerequisites": lambda **kwargs: df_to_html(procedure("GetCoursePrerequisites",(kwargs["dept_code"],kwargs["crs_num"]))),
    "What is the average course depth and number of prerequisites for each department?": lambda **kwargs: df_to_html(query(avg_crs_depth_prereq)),
    "What are the five courses with the lowest passing rate?": lambda **kwargs: df_to_html(query(five_lowest_passing_rate)),
    "Who are the five hardest professors?": lambda **kwargs: df_to_html(query(five_hardest_prof)),
    "What is the relationship between student grades and TA frequency?": lambda **kwargs: df_to_html(query(rel_grade_TA)),
    "What is the average student GPA for each major?": lambda **kwargs: df_to_html(query(avg_gpa_by_major)),
    "How does student performance change across quarters?": lambda **kwargs: df_to_html(query(stud_perf_by_quarter))
}

options = list(opt_fnc.keys())

In [7]:
def submit(option_input,sql,stud_id,prof_id,year,quarter,dept_code,crs_num):
    # If integer parameters are submitted, make sure that they are acutally integers
    if stud_id != "":
        try:
            stud_id = int(stud_id)
        except ValueError:
            return "Student id must be an integer"
    if prof_id != "":
        try:
            prof_id = int(prof_id)
        except ValueError:
            return "Professor id must be an integer"
    if year != "":
        try:
            year = int(year)
        except ValueError:
            return "Year must be an integer"
    if quarter != "":
        try:
            quarter = int(quarter)
        except ValueError:
            return "Student id must be an integer"
    if crs_num != "":
        try:
            crs_num = int(crs_num)
        except ValueError:
            return "Course number must be an integer"
    
    # Pass the parameter data to the corresponding function
    kwargs = {
        "sql": sql,
        "stud_id": stud_id,
        "prof_id": prof_id,
        "year": year,
        "quarter": quarter,
        "dept_code": dept_code,
        "crs_num": crs_num
    }
    return opt_fnc[option_input](**kwargs)

# returns a function lambda(option) that toggles visibility if option is in opts
def update_visibility(opts):
    return lambda option: gr.update(visible=(option in opts))

with gr.Blocks() as demo:
    option_input = gr.Radio(options, label = "Select an option")

    # parameter fields
    sql = gr.Textbox(label="Enter SQL", visible=False)
    stud_id = gr.Textbox(label="Enter Student ID", visible=False)
    prof_id = gr.Textbox(label="Enter Professor ID", visible=False)
    year = gr.Textbox(label="Enter Year", visible=False)
    quarter = gr.Textbox(label="Enter Quarter (1,2,3,4)", visible=False)
    dept_code = gr.Textbox(label="Enter Department Code", visible=False)
    crs_num = gr.Textbox(label="Enter Course Number", visible=False)

    # update parameter visibility if the corresponding options are selected
    option_input.change(update_visibility([options[0]]), option_input, sql)
    option_input.change(update_visibility([options[1]]), option_input, stud_id)
    option_input.change(update_visibility([options[2]]), option_input, prof_id)
    option_input.change(update_visibility([options[2]]), option_input, year)
    option_input.change(update_visibility([options[2]]), option_input, quarter)
    option_input.change(update_visibility([options[3]]), option_input, dept_code)
    option_input.change(update_visibility([options[3]]), option_input, crs_num)

    submit_button = gr.Button("Submit")
    output = gr.HTML()
    submit_button.click(
        submit,
        inputs = [option_input,sql,stud_id,prof_id,year,quarter,dept_code,crs_num],
        outputs = output
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
